In [13]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate, 
    ChatPromptTemplate
)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [14]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [15]:
# FewShotPromptTemplate을 사용하여 뉴스 기사에서 핵심 키워드 3개를 추출하는 시스템을 구현해보세요.
# 주어진 예시들을 참고하여 일관된 형식으로 키워드를 추출해야 합니다.

# 3) 예시 데이터
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "대한민국 축구 대표팀이 카타르 월드컵 최종예선에서 강력한 상대인 이란을 상대로 극적인 승리를 거두며 본선 진출 가능성을 높였다. 손흥민 선수의 결승골이 결정적이었다.",
        "keywords": "대한민국축구, 카타르월드컵, 손흥민",
    },
    {
        "news": "정부는 최근 급증하는 물가 안정을 위해 긴급 경제 대책을 발표했다. 주요 내용으로는 유류세 인하 연장과 농축수산물 할인 지원 확대 등이 포함되었다.",
        "keywords": "물가안정, 경제대책, 유류세인하",
    }
]


In [16]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000024B45B52900> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000024B4588E210> root_client=<openai.OpenAI object at 0x0000024B45B532C0> root_async_client=<openai.AsyncOpenAI object at 0x0000024B45807230> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [17]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

In [18]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [ ]:
# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])

In [ ]:
input_news = "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio)\
            및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. \
            모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech)\
            및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."

In [21]:
chain = final_prompt | llm
result = chain.invoke({"input": input_news})
print(result.content)

키워드: 제미나이, 구글 AI, 멀티모달
